In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
all_files = glob.glob('dataset' + '/*.csv')

In [3]:
lst = []
for filename in all_files:
    data = pd.read_csv(filename, index_col=None, header=0)
    lst.append(data)

In [4]:
df = pd.concat(lst, axis=0, ignore_index=True)

In [5]:
df = df[~df['Identifier'].str.contains(r'[0-9]')]

In [7]:
df.head()

,Date,Category1,Category2,Category3,Maker,Sales,Identifier
0,2010-02-15,Air,Conditioners,NaN,Toshino,2143.0,RVFWTWSMNU
2,2010-02-15,Washers,&,Dryers,Akaza,3515.0,VUQUQMHODG
3,2010-02-15,Refrigerators,NaN,NaN,Akaza,687.0,XVIOZZOUTN
4,2010-02-15,Washers,&,Dryers,Shimada,435.0,RBRMDIXXGL
6,2010-02-15,Refrigerators,NaN,NaN,Funami,726.0,BIBIOOIHA


In [8]:
df['Category'] = np.nan

bool_cat2isnull_cat3isnull = (df['Category2'].isnull()) & (df['Category3'].isnull())
bool_cat2notnull_cat3isnull = (df['Category2'].notnull()) & (df['Category3'].isnull())
bool_cat2notnull_cat3notnull = (df['Category2'].notnull()) & (df['Category3'].notnull())

df['Category'] = df['Category'].mask(bool_cat2isnull_cat3isnull, df['Category1'])
df['Category'] = df['Category'].mask(bool_cat2notnull_cat3isnull, df['Category1'] + ' ' + df['Category2'])
df['Category'] = df['Category'].mask(bool_cat2notnull_cat3notnull, df['Category1'] + ' ' + df['Category2'] + ' ' + df['Category3'])

df = df.drop(['Category1', 'Category2', 'Category3'], axis=1)

In [9]:
df.head()

,Date,Maker,Sales,Identifier,Category
0,2010-02-15,Toshino,2143.0,RVFWTWSMNU,Air Conditioners
2,2010-02-15,Akaza,3515.0,VUQUQMHODG,Washers & Dryers
3,2010-02-15,Akaza,687.0,XVIOZZOUTN,Refrigerators
4,2010-02-15,Shimada,435.0,RBRMDIXXGL,Washers & Dryers
6,2010-02-15,Funami,726.0,BIBIOOIHA,Refrigerators


In [10]:
df

,Date,Maker,Sales,Identifier,Category
0,2010-02-15,Toshino,2143.0,RVFWTWSMNU,Air Conditioners
2,2010-02-15,Akaza,3515.0,VUQUQMHODG,Washers & Dryers
3,2010-02-15,Akaza,687.0,XVIOZZOUTN,Refrigerators
4,2010-02-15,Shimada,435.0,RBRMDIXXGL,Washers & Dryers
6,2010-02-15,Funami,726.0,BIBIOOIHA,Refrigerators
...,...,...,...,...,...
134956,2021-04-29,Shimada,1258.0,WVJTPVZKAQ,Ovens & Cooktops
134960,2021-04-29,Akaza,5422.0,VUQUQMHODG,Washers & Dryers
134961,2021-04-29,Hayashida,539.0,GCPKFKSEWG,Vacuum Cleaners
134962,2021-04-29,Furutani,10547.0,FVDHIZGQRC,Water Pumps


In [ ]:
lst = []
for filename in all_files:
    # Read the data
    data = pd.read_csv(filename, index_col=None, header=0)
    
    # Fill the missing Sales data
    grpby_id = data.groupby(by='Identifier').mean().round(2)
    for identifier, sales_mean in zip(grpby_id.index, grpby_id.values):
        data['Sales'] = data['Sales'].mask((data['Sales'].isnull()) & (data['Identifier'] == identifier), sales_mean)
    
    # Convert Date to datetime
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Exclude certain Identifiers
    data = data[~data['Identifier'].str.contains(r'[0-9]')]
    
    # Groupby Date and set the freq index to 'D' (day)
    by_date = data.groupby(by='Date').sum()
    
    lst.append(by_date)

In [11]:
by_date = df.groupby(by='Date').sum()

In [12]:
by_date

,Sales
Date,
2010-02-15,58706.0
2010-02-16,90843.0
2010-02-17,90227.0
2010-02-18,90822.0
2010-02-19,89663.0
...,...
2021-04-23,142664.0
2021-04-26,134923.0
2021-04-27,139043.0


In [13]:
by_date.loc['2021-01-01':]

,Sales
Date,
2021-01-04,156022.0
2021-01-05,161050.0
2021-01-06,159184.0
2021-01-07,152695.0
2021-01-08,163192.0
...,...
2021-04-23,142664.0
2021-04-26,134923.0
2021-04-27,139043.0


In [14]:
by_date.loc['2021-01-01':].sum()

Sales    11849286.0
dtype: float64

In [15]:
s2021 = 11849286
fcast_mean = 142239.41

In [16]:
30000000 - 11849286

18150714

In [17]:
18150714 / fcast_mean

127.60678633298605

In [18]:
total_days = 128

In [23]:
from datetime import datetime, timedelta
s = '2021-04-29'
date = datetime.strptime(s, '%Y-%m-%d')

In [24]:
date

datetime.datetime(2021, 4, 29, 0, 0)

In [25]:
modified_date = date + timedelta(days=total_days)

In [26]:
modified_date

datetime.datetime(2021, 9, 4, 0, 0)

In [27]:
datetime.strftime(modified_date, '%Y-%m-%d')

'2021-09-04'